In [3]:
import numpy as np

In [4]:
import re

def parse_views(text):
    match = re.search(r'([\d.,]+)\s*([KMB]?)', text, re.IGNORECASE)
    if not match:
        return None

    number, suffix = match.groups()
    number = float(number.replace(',', '.'))

    multipliers = {'K': 1000, 'M': 1000000, 'B': 1000000000}
    multiplier = multipliers.get(suffix.upper(), 1)

    return int(number * multiplier)

In [6]:
import json
import pandas as pd

with open("danzan_girlfriends.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame.from_dict(data, orient="index")

df.reset_index(inplace=True)
df.rename(columns={"index": "name"}, inplace=True)

df["views"] = df["views"].apply(parse_views).astype(int)
df["video_counts"] = df["video_counts"].str[:-6].astype(int)
df["name"] = df["name"].astype(str)

for col in ["twitter", "instagram", "onlyfans", "tiktok", "facebook"]:
    df[col + "_exists"] = df[col].notna()

print(df)

                   name       views  video_counts  \
0            Alex Adams  3500000000           491   
1          Lana Rhoades  2000000000           409   
2          ANGELA WHITE  1700000000          1028   
3          Violet Myers   771000000           392   
4           Johnny Sins  2100000000           877   
...                 ...         ...           ...   
11278         harumeiji      603000             6   
11279          Janner3D     6400000            81   
11280            Reebek       96700             9   
11281      Hide_Fantasy     7600000            46   
11282  ExtendedMarriage           0             0   

                                    twitter  \
0                                       NaN   
1       https://www.twitter.com/lanarhoades   
2       https://www.twitter.com/angelawhite   
3           https://twitter.com/violetsaucy   
4         http://www.twitter.com/JohnnySins   
...                                     ...   
11278  https://www.twitter.com/Ero

In [12]:
print(sum(df["twitter_exists"]))
print(sum(df["instagram_exists"]))
print(sum(df["onlyfans_exists"]))
print(sum(df["tiktok_exists"]))
print(sum(df["facebook_exists"]))

6722
5921
2860
1572
15
